In [1]:
!nvidia-smi

Sat Apr 13 21:38:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA H100 80GB HBM3          On  | 00000000:87:00.0 Off |                    0 |
| N/A   35C    P0              76W / 700W |      4MiB / 81559MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
# !wget https://people.eecs.berkeley.edu/~hendrycks/data.tar -O mmlu.tar
# !tar -xf mmlu.tar -C mmlu 

In [3]:
# This will reload the imported modules (e.g. get_decode_model_characterstics) every time you execute the jupyter cells, so that you don't need to restart the notebook after updating the source codes.
%load_ext autoreload
%autoreload 2  

In [4]:

import os, sys
script_dir = os.getcwd()
module_path = script_dir
for _ in range(1):
    module_path = os.path.abspath(os.path.join(module_path, '../'))
    if module_path not in sys.path:
        sys.path.insert(0,module_path)
        
sys.path.append("mixtral-offloading")
import torch
from torch.nn import functional as F
from hqq.core.quantize import BaseQuantizeConfig
from huggingface_hub import snapshot_download
from IPython.display import clear_output
from tqdm.auto import trange
from transformers import AutoConfig, AutoTokenizer
from transformers.utils import logging as hf_logging
import time
import gc
from src.build_model import OffloadConfig, QuantConfig, build_model

hqq_aten package not installed. HQQBackend.ATEN backend will not work unless you install the hqq_aten lib in hqq/kernels.


/scratch/584752/moe_offload/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
quantized_model_name = "lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo"
state_path = "Mixtral-8x7B-Instruct-v0.1-offloading-demo2"

config = AutoConfig.from_pretrained(quantized_model_name)

device = torch.device("cuda:0")

##### Change this to 5 if you have only 12 GB of GPU VRAM #####
offload_per_layer = 4
# offload_per_layer = 5
###############################################################

num_experts = config.num_local_experts

offload_config = OffloadConfig(
    main_size=config.num_hidden_layers * (num_experts - offload_per_layer),
    offload_size=config.num_hidden_layers * offload_per_layer,
    buffer_size=4,
    offload_per_layer=offload_per_layer,
)


attn_config = BaseQuantizeConfig(
    nbits=4,
    group_size=64,
    quant_zero=True,
    quant_scale=True,
)
attn_config["scale_quant_params"]["group_size"] = 256


ffn_config = BaseQuantizeConfig(
    nbits=2,
    group_size=16,
    quant_zero=True,
    quant_scale=True,
)
quant_config = QuantConfig(ffn_config=ffn_config, attn_config=attn_config)


# del model

gc.collect
torch.cuda.empty_cache()


In [24]:

model = build_model(
    device=device,
    quant_config=quant_config,
    offload_config=offload_config,
    state_path=state_path,
    routing_strategy="THRESHOLDING",
    routing_threshold=0.05
)

/scratch/584752/moe_offload/lib/python3.10/site-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
Loading experts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:04<00:00,  7.13it/s]


In [25]:
from mmlu import test_mmlu
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define benchmark with specific tasks and shots
test_mmlu(model_name=model_name, model_loaded=model, tokenizer=tokenizer, data_dir="./mmlu", save_dir="./thresholding_results")

Starting abstract_algebra, dev size:(5, 6), Test size:(100, 6)
Average accuracy 0.340 , Average Time:1.175 sec, avg expert load reduced: 415.18, - abstract_algebra
Starting anatomy, dev size:(5, 6), Test size:(135, 6)
Average accuracy 0.593 , Average Time:1.159 sec, avg expert load reduced: 1290.0814814814814, - anatomy
Starting astronomy, dev size:(5, 6), Test size:(152, 6)
Average accuracy 0.789 , Average Time:1.400 sec, avg expert load reduced: 2323.8355263157896, - astronomy
Starting business_ethics, dev size:(5, 6), Test size:(100, 6)
Average accuracy 0.670 , Average Time:1.383 sec, avg expert load reduced: 3235.99, - business_ethics
Starting clinical_knowledge, dev size:(5, 6), Test size:(265, 6)
Average accuracy 0.758 , Average Time:1.235 sec, avg expert load reduced: 4516.954716981132, - clinical_knowledge
Starting college_biology, dev size:(5, 6), Test size:(144, 6)
Average accuracy 0.764 , Average Time:1.293 sec, avg expert load reduced: 5972.770833333333, - college_biology
S

In [ ]:
# Result Ref: https://wandb.ai/byyoung3/ml-news/reports/Testing-Mixtral-8x7B-with-MMLU-and-W-B---Vmlldzo2MjI0ODAz

In [26]:

model = build_model(
    device=device,
    quant_config=quant_config,
    offload_config=offload_config,
    state_path=state_path,
    routing_strategy="THRESHOLDING",
    routing_threshold=0.075
)

/scratch/584752/moe_offload/lib/python3.10/site-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
Loading experts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:04<00:00,  6.86it/s]


In [ ]:
from mmlu import test_mmlu
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define benchmark with specific tasks and shots
test_mmlu(model_name=model_name, model_loaded=model, tokenizer=tokenizer, data_dir="./mmlu", save_dir="./thresholding_0_075_results")

Starting abstract_algebra, dev size:(5, 6), Test size:(100, 6)
Average accuracy 0.340 , Average Time:1.133 sec, avg expert load reduced: 599.33, - abstract_algebra
Starting anatomy, dev size:(5, 6), Test size:(135, 6)
Average accuracy 0.593 , Average Time:1.144 sec, avg expert load reduced: 1882.8592592592593, - anatomy
Starting astronomy, dev size:(5, 6), Test size:(152, 6)
Average accuracy 0.789 , Average Time:1.384 sec, avg expert load reduced: 3385.125, - astronomy
Starting business_ethics, dev size:(5, 6), Test size:(100, 6)
Average accuracy 0.670 , Average Time:1.366 sec, avg expert load reduced: 4655.49, - business_ethics
Starting clinical_knowledge, dev size:(5, 6), Test size:(265, 6)
Average accuracy 0.758 , Average Time:1.219 sec, avg expert load reduced: 6464.0037735849055, - clinical_knowledge
Starting college_biology, dev size:(5, 6), Test size:(144, 6)
Average accuracy 0.764 , Average Time:1.277 sec, avg expert load reduced: 8545.583333333334, - college_biology
Starting c